# Create Venue Location map

# Analysis of University Metrics

This notebook shows, how our graphs enables the analysis of the geographic features of venues. Here, we create a map of where universities publications are published in the world. This can be used to assess the international relevance of an university.

In [1]:
from graph.oa_graph import OpenAlexGraph
from graph.analyse_graph import UNI_QUERY_LIST

g = OpenAlexGraph()
g.parse("out/graph.ttl")

<Graph identifier=N8a4744002d15475c8759836fb197a65e (<class 'graph.oa_graph.OpenAlexGraph'>)>

In [2]:
q = f"""
    SELECT
        (SAMPLE(?inst_name) as ?INST)
        (CONCAT('[', GROUP_CONCAT(CONCAT('(', str(?lat), ', ', str(?lng), ')'); separator=', '), ']') as ?COORD)
        (CONCAT('[', GROUP_CONCAT(CONCAT('"', ?ven_name, '"'); separator=', '), ']') as ?VENUE)
    WHERE {{
        ?id_inst a schema:EducationalOrganization ;
            schema:name ?inst_name .

        VALUES ?inst_name {{ {UNI_QUERY_LIST} }}

        ?work dbp:institution ?id_inst ;
            schema:event [
                schema:name ?ven_name ;
                schema:location [
                    schema:latitude ?lat ;
                    schema:longitude ?lng
                ]
            ]
    }}
    GROUP BY ?inst_name
"""
res = g.query(q)
res

In [14]:
import folium
import webbrowser
from folium.plugins import MarkerCluster, HeatMap

folium_map = folium.Map(location=[20, 0], max_bounds=True, zoom_start=2, tiles='CartoDB positron')

for row in res:
    inst = row.asdict()["INST"]
    coords = eval(row.asdict()["COORD"])
    venue = eval(row.asdict()["VENUE"])

    layer = folium.FeatureGroup(name=inst, show=False)
    cluster = MarkerCluster(locations=coords, popups=venue).add_to(layer)
    # HeatMap(coords).add_to(layer)
    folium_map.add_child(layer)

folium_map.add_child(folium.map.LayerControl())
folium_map.save("map.html")
webbrowser.open("map.html")

True